In [ ]:
import os
!git clone https://github.com/mohammed-elkomy/two-stream-action-recognition.git
os.chdir("/content/two-stream-action-recognition")

!git clone https://github.com/circulosmeos/gdown.pl.git
!./gdown.pl/gdown.pl https://drive.google.com/file/d/1djGzpxAYFvNX-UaQ7ONqDHGgnzc8clBK/view "spatial.zip" 
!./gdown.pl/gdown.pl https://drive.google.com/file/d/1kvslNL8zmZYaHRmhgAM6-l_pNDDA0EKZ/view "motion.zip"
!unzip spatial.zip
!unzip motion.zip

!pip install -U -q PyDrive 2> s.txt >> s.txt
!pip install opencv-python 2> s.txt >> s.txt
!pip install imgaug 2> s.txt >> s.txt
!pip install scikit-video 2> s.txt >> s.txt

In [ ]:
%cat ./UCF_list/classInd.txt

In [ ]:
import cv2
from imgaug import augmenters as iaa

from evaluation import legacy_load_model
from evaluation.evaluation import *

import random
from frame_dataloader import DataUtil

import matplotlib.pyplot as plt
import numpy as np

import skvideo.io

import io
import base64
from IPython.display import HTML

from matplotlib import gridspec

In [ ]:
#Category mapping - 101 into n categories
import csv
import collections
filepath = '/content/two-stream-action-recognition/drive/My Drive/adplace/actionoutput.csv'
mappingDict = collections.OrderedDict()
mappingDict = {
    "ApplyEyeMakeup" : 1,
    "ApplyLipstick" : 1,
    "Archery" : 2,
    "BabyCrawling" : 1,
    "BalanceBeam" : 2,
    
    "BandMarching" : 2,
    "BaseballPitch" : 3,
    "Basketball" : 3,
    "Basketball" : 3,
    "BenchPress" : 2,
    
    "Biking" : 3,
    "Billiards" : 2,
    "BlowDryHair" : 1,
    "BlowingCandles" : 1,
    "BodyWeightSquats" : 1,
    
    "Bowling" : 2,
    "BoxingPunchingBag" : 2,
    "BoxingSpeedBag" : 2,
    "BreastStroke" : 3,
    "BrushingTeeth" : 1,
    
    "CleanAndJerk" : 1,
    "CliffDiving" : 3,
    "CricketBowling" : 2,
    "CricketShot" : 3,
    "CuttingInKitchen" : 1,
    
    "Diving" : 3,
    "Drumming" : 3,
    "Fencing" : 1,
    "FieldHockeyPenalty" : 3,
    "FloorGymnastics" : 3,
    
    "FrisbeeCatch" : 2,
    "FrontCrawl" : 1,
    "GolfSwing" : 2,
    "Haircut" : 1,
    "Hammering" : 1,
    
    "HammerThrow" : 1,
    "HandstandPushups" : 2,
    "HandstandWalking" : 3,
    "HeadMassage" : 1,
    "HighJump" : 2,
    
    "HorseRace" : 3,
    "HorseRiding" : 3,
    "HulaHoop" : 2,
    "IceDancing" : 3,
    "JavelinThrow" : 2,
    
    "JugglingBalls" : 3,
    "JumpingJack" : 3,
    "JumpRope" : 2,
    "Kayaking" : 3,
    "Knitting" : 1,
    
    "LongJump" : 2,
    "Lunges" : 2,
    "MilitaryParade" : 2,
    "Mixing" : 1,
    "MoppingFloor" : 1,
    
    "Nunchucks" : 2,
    "ParallelBars" : 2,
    "PizzaTossing" : 2,
    "PlayingCello" : 2,
    "PlayingDaf" : 2,
    
    "PlayingDhol" : 3,
    "PlayingFlute" : 3,
    "PlayingGuitar" : 3,
    "PlayingPiano" : 3,
    "PlayingSitar" : 2,
    
    "PlayingTabla" : 2,
    "PlayingViolin" : 2,
    "PoleVault" : 2,
    "PommelHorse" : 2,
    "PullUps" : 2,
    
    "Punch" : 2,
    "PushUps" : 2,
    "Rafting" : 3,
    "RockClimbingIndoor" : 3,
    "RopeClimbing" : 3,
    
    "Rowing" : 3,
    "SalsaSpin" : 3,
    "ShavingBeard" : 1,
    "Shotput" : 2,
    "SkateBoarding" : 3,
    
    "Skiing" : 3,
    "Skijet" : 3,
    "SkyDiving" : 3,
    "SoccerJuggling" : 3,
    "SoccerPenalty" : 3,
    
    "StillRings" : 2,
    "SumoWrestling" : 2,
    "Surfing" : 3,
    "Swing" : 2,
    "TableTennisShot" : 3,
    
    "TaiChi" : 2,
    "TennisSwing" : 3,
    "ThrowDiscus" : 2,
    "TrampolineJumping" : 3,
    "Typing" : 1,
    
    "UnevenBars" : 1,
    "VolleyballSpiking" : 2,
    "WalkingWithDog" : 2,
    "WallPushups" : 2,
    "WritingOnBoard" : 1,
    
    "YoYo" : 2
}
category_list_in_order = [1,1,2,1,2,2,3,3,3,2,3,2,1,1,1,2,2,2,3,1,1,3,2,3,1,3,3,1,3,3,2,1,2,1,1,1,2,3,1,2,3,3,2,3,2,3,3,2,3,1,2,2,2,1,1,2,2,2,2,2,2,2,3,3,3,3,3,1,2,3,3,3,3,3,3,2,2,3,2,3,2,3,2,3,1,1,2,2,2,1,2]


In [ ]:
def getValueFromKey(key):
  return mappingDict[key]

def writeToCSV(filename, category):
  with open(filepath, 'a') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow([filename, category])
    
def postLstmOps(filename, action):
  category = getValueFromList(action)
  writeToCSV(filename, category)
 

def getValueFromList(index):
  return category_list_in_order[index]
  

In [ ]:
# defining functions and global objects

# dictionary of class names
data_util = DataUtil(path= './UCF_list/', split="01")
action_names =  {v:k for k,v in data_util.action_to_label.items()} # class name dictionary
print (action_names)
stacked_frames = 10

# image resize augmenter to be fed into the network
augmenter = iaa.Sequential([
    iaa.Resize({"height": 299, "width": 299})
])


def convert_to_image(flow_image):
    """
    this is the conversion function of each flow frame
    based on the cpp version of extracting optical flow https://github.com/feichtenhofer/gpu_flow/blob/master/compute_flow.cpp
    """
    l, h = -20, 20
    return (255 * (flow_image - l) / (h - l)).astype(np.uint8)


def stack_opticalflow(start_frame_index, stacked_frames):  # returns numpy (h,w,stacked*2) = one sample
    """
    Stacks "stacked_frames" u/v frames on a single numpy array : (h,w,stacked*2)
    """
    first_optical_frame_u = original_u_frames[start_frame_index]  # horizontal
    first_optical_frame_v = original_v_frames[start_frame_index]  # vertical

    stacked_optical_flow_sample = np.zeros(first_optical_frame_u.shape + (2 * stacked_frames,), dtype=np.uint8)  # with channel dimension of  stacked_frames(u)+ stacked_frames(v)

    stacked_optical_flow_sample[:, :, 0] = first_optical_frame_u
    stacked_optical_flow_sample[:, :, 0 + stacked_frames] = first_optical_frame_v

    for index, optical_frame_id in enumerate(range(start_frame_index + 1, start_frame_index + stacked_frames), 1):  # index starts at 1 placed after the first one
        stacked_optical_flow_sample[:, :, index] = original_u_frames[optical_frame_id]
        stacked_optical_flow_sample[:, :, index + stacked_frames] = original_v_frames[optical_frame_id]

    return stacked_optical_flow_sample


def get_image_from_fig(fig):
    """
    converts matplotlib figure into a numpy array for demo video generation
    """
    fig.canvas.draw()

    data = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8, sep='')
    data = data.reshape(fig.canvas.get_width_height()[::-1] + (3,))

    return data

In [ ]:

# legacy_load_model is an older version of keras load_model since keras API changed a little bit when I was working on action recognition 

# load into ram
print("Loading Spatial stream")
spatial_model_restored = legacy_load_model(filepath="spatial.h5", custom_objects={'sparse_categorical_cross_entropy_loss': sparse_categorical_cross_entropy_loss, "acc_top_1": acc_top_1, "acc_top_5": acc_top_5})
spatial_model_restored.summary()


# load into ram
print("Loading Motion stream")
motion_model_restored = legacy_load_model(filepath="motion.h5", custom_objects={'sparse_categorical_cross_entropy_loss': sparse_categorical_cross_entropy_loss, "acc_top_1": acc_top_1, "acc_top_5": acc_top_5})
motion_model_restored.summary()

In [ ]:
from google.colab import drive
drive.mount('drive')

In [ ]:
%ls '/content/two-stream-action-recognition/drive/My Drive/adplace/scene_videos'

In [ ]:
import os
# select a random video
video_dir="/content/two-stream-action-recognition/drive/My Drive/adplace/scene_videos/"
videos = os.listdir(video_dir)
print(videos)
for video in videos:
  video_path = video_dir+video
  print("selected_video:",video_path)
  vidcap = cv2.VideoCapture(selected_video)
  print("frame rate for demo:",vidcap.get(cv2.CAP_PROP_FPS))
  success, image = vidcap.read()
  # make the rgb frames
  original_rgb_frames = []

  while success:
      original_rgb_frames.append(image)
      success, image = vidcap.read()

  print("frames count in video", len(original_rgb_frames))

  # make the optical flow frames
  original_v_frames = []
  original_u_frames = []

  frames = list(map(lambda frame: cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY).astype(np.float32) / 255.0, original_rgb_frames))
  optical_flow = cv2.DualTVL1OpticalFlow_create()

  for frame_index in range(len(frames) - 1):
      if frame_index % 50 == 0:
          print("processing tvl flow of frame ",frame_index)

      flow = optical_flow.calc(frames[frame_index], frames[frame_index + 1], None)
      u_frame = convert_to_image(flow[..., 0])
      v_frame = convert_to_image(flow[..., 1])

      original_v_frames.append(v_frame)
      original_u_frames.append(u_frame)

  print("original_rgb_frames:", len(original_rgb_frames), "original_u_frames:", len(original_u_frames), "original_v_frames:", len(original_v_frames))
  # generate spatial batch as done in the dataloader
  spatial_batch = []
  for image in original_rgb_frames:
      spatial_batch.append(
          cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
      )

  spatial_batch = np.array(augmenter.augment_images(spatial_batch), dtype=np.float32) / 255.0

  # generate motion batch as done in the dataloader
  motion_batch = []

  for first_optical_frame_id in range(len(original_u_frames) - stacked_frames):
      motion_batch.append(  # append one sample which is (h,w,stacked*2)
          stack_opticalflow(start_frame_index=first_optical_frame_id, stacked_frames=stacked_frames)
      )
  motion_batch = np.array(augmenter.augment_images(motion_batch), dtype=np.float32) / 255.0

  """
  predict spatial stream output
  """
  spatial_pred = spatial_model_restored.predict(spatial_batch)
  spatial_classes = np.argsort(spatial_pred,axis=1)[:,:-6:-1]
  spatial_scores = np.sort(spatial_pred,axis=1)[:,:-6:-1]
  """
  predict motion stream output
  """
  motion_pred = motion_model_restored.predict(motion_batch)
  motion_classes = np.argsort(motion_pred,axis=1)[:,:-6:-1]
  motion_scores = np.sort(motion_pred,axis=1)[:,:-6:-1]
  """
  get the average output prediction
  """
  average_pred = motion_pred + spatial_pred[:motion_pred.shape[0],]
  average_classes = np.argsort(average_pred,axis=1)[:,:-6:-1]
  average_scores = np.sort(average_pred,axis=1)[:,:-6:-1]

  final_class = []
  # generating output video
  for frame_index in range(motion_classes.shape[0]): 
    if frame_index % 50 == 0:
        print("processing frame ",frame_index)
    final_class.append(average_classes[frame_index][0])

  print(final_class)
  class_dict = {}
  for frame_class in final_class:
    class_dict[frame_class] = class_dict.get(frame_class, 0) + 1
  final_class_index = max(class_dict, key=class_dict.get)
  print(final_class_index)
  postLstmOps(video,final_class_index)